# CS246 - Colab 1
## Word Count in Spark

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=47415c3f57d3cae8a8080aa77f0599e3d70dec8aa6217f1b947ab3dc71a6ab49
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:a

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you successfully run the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter, we want to count the total number of (non-unique) words that start with a specific letter.

In your implementation, **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all words that **start** with a non-alphabetic character. You should output word counts for the **entire document**, inclusive of the title, author, and the main texts. If you encounter words broken as a result of new lines, e.g. "pro-ject" where the segment after the dash sign is on a new line, no special processing is needed and you can safely consider it as two words.

Your outputs will be graded on a range -- if your differences from the ground-truths are within an error threshold of 5, you'll be considered correct.

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [5]:
# YOUR
txt = spark.read.text("pg100.txt")
txt.show()

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
| William Shakespeare|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|** This is a COPY...|
|**     Please fol...|
|                    |
|Title: The Comple...|
|                    |
|Author: William S...|
|                    |
|Posting Date: Sep...|
|Release Date: Jan...|
|                    |
|   Language: English|
|                    |
+--------------------+
only showing top 20 rows



In [107]:
def clean_text(text):
  words = []
  word = text.lower().split()
  for w in word:
    if w[0].isalpha():
      for char in w.strip(".,: ").split('-'):
        if char.isalpha():
          words += [(char[0],1)]
  return words

In [109]:
word_list = txt.rdd.flatMap(lambda row: clean_text(row.value))
word_list.take(3)

[('t', 1), ('p', 1), ('g', 1)]

In [111]:
word_count = word_list.reduceByKey(lambda a, b:a+b).sortBy(lambda x: x[0])
word_count.take(26)

[('a', 82183),
 ('b', 43043),
 ('c', 31121),
 ('d', 26877),
 ('e', 17388),
 ('f', 33988),
 ('g', 19245),
 ('h', 55904),
 ('i', 58451),
 ('j', 2929),
 ('k', 8445),
 ('l', 26492),
 ('m', 51762),
 ('n', 25133),
 ('o', 41570),
 ('p', 24898),
 ('q', 2169),
 ('r', 12574),
 ('s', 60431),
 ('t', 117619),
 ('u', 8446),
 ('v', 5211),
 ('w', 56008),
 ('x', 14),
 ('y', 24662),
 ('z', 64)]

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!